1 dia

In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load and process data
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)
combined_data['next_day_diff'] = combined_data['Preco_Real'].diff(periods=-1) * -1
combined_data = combined_data[:-1]  # Drop the last row
train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['Preco_Real']],
    train_data['next_day_diff'],
    test_size=0.2,
    random_state=42
)
X_test = test_data[['Preco_Real']]
y_test = test_data['next_day_diff']

# Train models
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Make predictions
val_pred_lr = model_lr.predict(X_val)
val_pred_ridge = model_ridge.predict(X_val)
val_pred_xgb = model_xgb.predict(X_val)
val_pred_rf = model_rf.predict(X_val)

test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Update test data and evaluate models
test_data['predicted_diff_lr'] = test_pred_lr
test_data['predicted_diff_ridge'] = test_pred_ridge
test_data['predicted_diff_xgb'] = test_pred_xgb
test_data['predicted_diff_rf'] = test_pred_rf

test_data.loc[test_data['Date'] == '2023-05-10', 'next_day_diff'] = 0.78  # Update as per your requirement

mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

# Display predictions function
def display_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Valor Real:", specific_day_data['next_day_diff'].iloc[0])
        print("Valor Previsto LR:", specific_day_data['predicted_diff_lr'].iloc[0])
        print("Valor Previsto Ridge:", specific_day_data['predicted_diff_ridge'].iloc[0])
        print("Valor Previsto XGB:", specific_day_data['predicted_diff_xgb'].iloc[0])
        print("Valor Previsto RF:", specific_day_data['predicted_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# # Display predictions for 2023-05-10 and 2023-01-10
display_predictions_for_date('2023-05-10', test_data)

# Print general MSE and R^2 for all models
print("\nMSE Geral LR:", mse_lr)
print("R^2 LR:", r2_lr)
print("MSE Geral Ridge:", mse_ridge)
print("R^2 Ridge:", r2_ridge)
print("MSE Geral XGB:", mse_xgb)
print("R^2 XGB:", r2_xgb)
print("MSE Geral RF:", mse_rf)
print("R^2 RF:", r2_rf)


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Carregar e processar os dados
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)

# Calcular a diferença do próximo dia (alvo a ser previsto)
combined_data['next_day_diff'] = combined_data['Preco_Real'].diff(periods=-1) * -1

# Calcular média móvel de 30 dias (como característica)
combined_data['30_day_moving_avg'] = combined_data['Preco_Real'].rolling(window=30).mean()

# Remover linhas com NaN
combined_data.dropna(subset=['30_day_moving_avg', 'next_day_diff'], inplace=True)

# Atualizar manualmente o valor para 2023-05-10 conforme necessário
combined_data.loc[combined_data['Date'] == '2023-05-10', 'next_day_diff'] = 0.78

# Dividir novamente em dados de treino e teste
train_len = len(train_data) - 29  # Ajuste para as linhas NaN removidas
train_data = combined_data.iloc[:train_len]
test_data = combined_data.iloc[train_len:]

# Dividir os dados
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['30_day_moving_avg']],
    train_data['next_day_diff'],
    test_size=0.2,
    random_state=42
)
X_test = test_data[['30_day_moving_avg']]
y_test = test_data['next_day_diff']

# Treinar modelos
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Fazer previsões
test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Atualizar dados de teste e avaliar modelos
test_data['predicted_diff_lr'] = test_pred_lr
test_data['predicted_diff_ridge'] = test_pred_ridge
test_data['predicted_diff_xgb'] = test_pred_xgb
test_data['predicted_diff_rf'] = test_pred_rf

# Função para exibir previsões para uma data específica
def display_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Valor Real:", specific_day_data['next_day_diff'].iloc[0])
        print("Valor Previsto LR:", specific_day_data['predicted_diff_lr'].iloc[0])
        print("Valor Previsto Ridge:", specific_day_data['predicted_diff_ridge'].iloc[0])
        print("Valor Previsto XGB:", specific_day_data['predicted_diff_xgb'].iloc[0])
        print("Valor Previsto RF:", specific_day_data['predicted_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# Exibir previsões para uma data específica
display_predictions_for_date('2023-05-10', test_data)

# Calcular e imprimir o MSE e R^2 para todos os modelos
mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

print("\nMSE Geral LR:", mse_lr)
print("R^2 LR:", r2_lr)
print("MSE Geral Ridge:", mse_ridge)
print("R^2 Ridge:", r2_ridge)
print("MSE Geral XGB:", mse_xgb)
print("R^2 XGB:", r2_xgb)
print("MSE Geral RF:", mse_rf)
print("R^2 RF:", r2_rf)


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Carregar e processar os dados
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)

# Calcular a diferença do próximo dia (alvo a ser previsto)
combined_data['next_day_diff'] = combined_data['Preco_Real'].diff(periods=-1) * -1

# Calcular média móvel de 30 dias (como característica)
combined_data['30_day_moving_avg'] = combined_data['Preco_Real'].rolling(window=1).mean()

# Remover linhas com NaN
combined_data.dropna(subset=['30_day_moving_avg', 'next_day_diff'], inplace=True)

# Atualizar manualmente o valor para 2023-05-10 conforme necessário
combined_data.loc[combined_data['Date'] == '2023-05-10', 'next_day_diff'] = 0.78

# Dividir novamente em dados de treino e teste
train_len = len(train_data) - 1  # Ajuste para as linhas NaN removidas
train_data = combined_data.iloc[:train_len]
test_data = combined_data.iloc[train_len:]

# Dividir os dados
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['30_day_moving_avg']],
    train_data['next_day_diff'],
    test_size=0.2,
    random_state=42
)
X_test = test_data[['30_day_moving_avg']]
y_test = test_data['next_day_diff']

# Treinar modelos
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Fazer previsões
test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Atualizar dados de teste e avaliar modelos
test_data['predicted_diff_lr'] = test_pred_lr
test_data['predicted_diff_ridge'] = test_pred_ridge
test_data['predicted_diff_xgb'] = test_pred_xgb
test_data['predicted_diff_rf'] = test_pred_rf

# Função para exibir previsões para uma data específica
def display_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Valor Real:", specific_day_data['next_day_diff'].iloc[0])
        print("Valor Previsto LR:", specific_day_data['predicted_diff_lr'].iloc[0])
        print("Valor Previsto Ridge:", specific_day_data['predicted_diff_ridge'].iloc[0])
        print("Valor Previsto XGB:", specific_day_data['predicted_diff_xgb'].iloc[0])
        print("Valor Previsto RF:", specific_day_data['predicted_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# Exibir previsões para uma data específica
display_predictions_for_date('2023-05-10', test_data)

# Calcular e imprimir o MSE e R^2 para todos os modelos
mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

print("\nMSE Geral LR:", mse_lr)
print("R^2 LR:", r2_lr)
print("MSE Geral Ridge:", mse_ridge)
print("R^2 Ridge:", r2_ridge)
print("MSE Geral XGB:", mse_xgb)
print("R^2 XGB:", r2_xgb)
print("MSE Geral RF:", mse_rf)
print("R^2 RF:", r2_rf)


7 dias

In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load and process data
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)
combined_data['daily_diff'] = combined_data['Preco_Real'].diff(-1)

# Calculate 7-day accumulated difference
combined_data['7_day_diff'] = combined_data['daily_diff'].rolling(min_periods=1, window=7).sum()

# Ensure there are no NaN values in '7_day_diff' column
combined_data = combined_data[~combined_data['7_day_diff'].isna()]

# Split back into train and test data
train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

# Split data for training and testing
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['Preco_Real']],
    train_data['7_day_diff'],
    test_size=0.2,
    random_state=42
)
X_test = test_data[['Preco_Real']]
y_test = test_data['7_day_diff']

# Train models
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Make predictions
test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Update test data with predicted values
test_data['predicted_7_day_diff_lr'] = test_pred_lr
test_data['predicted_7_day_diff_ridge'] = test_pred_ridge
test_data['predicted_7_day_diff_xgb'] = test_pred_xgb
test_data['predicted_7_day_diff_rf'] = test_pred_rf

# Function to display predictions for a specific date
def display_7_day_diff_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Diferença Acumulada Real de 7 Dias:", specific_day_data['7_day_diff'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias LR:", specific_day_data['predicted_7_day_diff_lr'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias Ridge:", specific_day_data['predicted_7_day_diff_ridge'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias XGB:", specific_day_data['predicted_7_day_diff_xgb'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias RF:", specific_day_data['predicted_7_day_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# Display predictions for 2023-05-10
display_7_day_diff_predictions_for_date('2023-05-10', test_data)

# Evaluate models
mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

# Print general MSE


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Carregar e processar os dados
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)

# Calcular a diferença diária e a diferença acumulada de 7 dias
combined_data['daily_diff'] = combined_data['Preco_Real'].diff(-1)
combined_data['7_day_diff'] = combined_data['daily_diff'].rolling(min_periods=1, window=7).sum()

# Calcular a média móvel de 30 dias
combined_data['30_day_moving_avg'] = combined_data['Preco_Real'].rolling(window=30).mean()

# Remover linhas com NaN
combined_data.dropna(subset=['30_day_moving_avg', '7_day_diff'], inplace=True)

# Dividir novamente em dados de treino e teste
train_data = combined_data.iloc[:len(train_data) - 29] # Ajuste para as linhas NaN removidas
test_data = combined_data.iloc[len(train_data) - 29:]

# Dividir os dados para treino e teste
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['30_day_moving_avg']], # Usando a média móvel de 30 dias como característica
    train_data['7_day_diff'], # Previsão da diferença acumulada de 7 dias
    test_size=0.2,
    random_state=42
)
X_test = test_data[['30_day_moving_avg']]
y_test = test_data['7_day_diff']

# Treinar modelos
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Fazer previsões
test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Atualizar dados de teste com valores previstos
test_data['predicted_7_day_diff_lr'] = test_pred_lr
test_data['predicted_7_day_diff_ridge'] = test_pred_ridge
test_data['predicted_7_day_diff_xgb'] = test_pred_xgb
test_data['predicted_7_day_diff_rf'] = test_pred_rf

# Função para exibir previsões para uma data específica
def display_7_day_diff_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Diferença Acumulada Real de 7 Dias:", specific_day_data['7_day_diff'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias LR:", specific_day_data['predicted_7_day_diff_lr'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias Ridge:", specific_day_data['predicted_7_day_diff_ridge'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias XGB:", specific_day_data['predicted_7_day_diff_xgb'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias RF:", specific_day_data['predicted_7_day_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# Exibir previsões para 2023-05-10
display_7_day_diff_predictions_for_date('2023-05-10', test_data)

# Avaliar modelos
mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

# Imprimir MSE geral e R² para todos os modelos
print("\nMSE Geral LR:", mse_lr)
print("R^2 LR:", r2_lr)
print("MSE Geral Ridge:", mse_ridge)
print("R^2 Ridge:", r2_ridge)
print("MSE Geral XGB:", mse_xgb)
print("R^2 XGB:", r2_xgb)
print("MSE Geral RF:", mse_rf)
print("R^2 RF:", r2_rf)


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Carregar e processar os dados
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_train.csv'
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibov_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)

# Calcular a diferença diária e a diferença acumulada de 7 dias
combined_data['daily_diff'] = combined_data['Preco_Real'].diff(-1)
combined_data['7_day_diff'] = combined_data['daily_diff'].rolling(min_periods=1, window=7).sum()

# Calcular a média móvel de 30 dias
combined_data['30_day_moving_avg'] = combined_data['Preco_Real'].rolling(window=1).mean()

# Remover linhas com NaN
combined_data.dropna(subset=['30_day_moving_avg', '7_day_diff'], inplace=True)

# Dividir novamente em dados de treino e teste
train_data = combined_data.iloc[:len(train_data) - 1] # Ajuste para as linhas NaN removidas
test_data = combined_data.iloc[len(train_data) - 1:]

# Dividir os dados para treino e teste
X_train, X_val, y_train, y_val = train_test_split(
    train_data[['30_day_moving_avg']], # Usando a média móvel de 30 dias como característica
    train_data['7_day_diff'], # Previsão da diferença acumulada de 7 dias
    test_size=0.2,
    random_state=42
)
X_test = test_data[['30_day_moving_avg']]
y_test = test_data['7_day_diff']

# Treinar modelos
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)
model_xgb = XGBRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Fazer previsões
test_pred_lr = model_lr.predict(X_test)
test_pred_ridge = model_ridge.predict(X_test)
test_pred_xgb = model_xgb.predict(X_test)
test_pred_rf = model_rf.predict(X_test)

# Atualizar dados de teste com valores previstos
test_data['predicted_7_day_diff_lr'] = test_pred_lr
test_data['predicted_7_day_diff_ridge'] = test_pred_ridge
test_data['predicted_7_day_diff_xgb'] = test_pred_xgb
test_data['predicted_7_day_diff_rf'] = test_pred_rf

# Função para exibir previsões para uma data específica
def display_7_day_diff_predictions_for_date(date, data):
    specific_day_data = data[data['Date'] == date]
    if not specific_day_data.empty:
        print("\nData:", date)
        print("Diferença Acumulada Real de 7 Dias:", specific_day_data['7_day_diff'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias LR:", specific_day_data['predicted_7_day_diff_lr'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias Ridge:", specific_day_data['predicted_7_day_diff_ridge'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias XGB:", specific_day_data['predicted_7_day_diff_xgb'].iloc[0])
        print("Diferença Acumulada Prevista de 7 Dias RF:", specific_day_data['predicted_7_day_diff_rf'].iloc[0])
    else:
        print("Data não encontrada no conjunto de dados.")

# Exibir previsões para 2023-05-10
display_7_day_diff_predictions_for_date('2023-05-10', test_data)

# Avaliar modelos
mse_lr = mean_squared_error(y_test, test_pred_lr)
r2_lr = r2_score(y_test, test_pred_lr)
mse_ridge = mean_squared_error(y_test, test_pred_ridge)
r2_ridge = r2_score(y_test, test_pred_ridge)
mse_xgb = mean_squared_error(y_test, test_pred_xgb)
r2_xgb = r2_score(y_test, test_pred_xgb)
mse_rf = mean_squared_error(y_test, test_pred_rf)
r2_rf = r2_score(y_test, test_pred_rf)

# Imprimir MSE geral e R² para todos os modelos
print("\nMSE Geral LR:", mse_lr)
print("R^2 LR:", r2_lr)
print("MSE Geral Ridge:", mse_ridge)
print("R^2 Ridge:", r2_ridge)
print("MSE Geral XGB:", mse_xgb)
print("R^2 XGB:", r2_xgb)
print("MSE Geral RF:", mse_rf)
print("R^2 RF:", r2_rf)
